# Module 22: Big Data Challenge - Amazon Reviews
## Home Improvement Reviews

In [ ]:
import os

# Using Spark Version 3.2.2. To find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [ ]:
#Connection to Postgres Database

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-26 09:19:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.57MB/s    in 0.2s    

2022-11-26 09:19:05 (5.57 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session with Postgres connection

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataPart1-HomeImprovement").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


## Extract

1. Connect to and load data from AWS to dataframe using PySpark
2. Include the headers from the dataset
3. Verify data and count of entries

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

# Modified since this is a tab delimited file.
review_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")

# Show Top entries of the DataFrame to be later dissected into the table schemas
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [ ]:
# State the size of the DF before dropping NAs
print(review_df.count())

2634781


## Transform

1. Remove NA from Dataframe
2. Remove duplicate entries
3. Rename columns (as needed) - Not Needed
4. Create separate dataframe for each table

In [ ]:
#Cleaning up the data set to drop all NAs and Duplicate values

# Drop NA values
dropna_df = review_df.dropna(how='any')

# State the size of the DF after dropping NAs
print(f"Size after dropping NA: {dropna_df.count()}")

# Dropping Duplicate values from the Dataframe
dropna_df = dropna_df.dropDuplicates()

# State the size of the DF after dropping duplicates
print(f"Size after removing any duplicates: {dropna_df.count()}")

Size after dropping NA: 2634255
Size after removing any duplicates: 2634255


In [ ]:
# Remove columns not included in schema. 
# These columns are not needed: marketplace, product_category, verified_purchase, review_headline, review_body

cleaned_df  = dropna_df.select(["customer_id","review_id","product_id", "product_parent","product_title","star_rating","helpful_votes","total_votes","vine","review_date"])
cleaned_df .show(5)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|    1297897|R100D3UT1KL2LP|B002N1U1FI|     414800992|Wallies 12238 Cam...|          5|            0|          0|   N| 2014-09-19|
|   25517286|R100VJOWSRD7MW|B000NCV2V6|     577225575|Heath Zenith BL-6...|          4|            0|          0|   N| 2013-11-24|
|   16192717|R103699E1EG79Y|B002TSN4SQ|     786521714|Master Lock 1500I...|          5|            0|          0|   N| 2015-06-01|
|    4583604|R1038R168TQ0M2|B007V4YAOK|     111253254|Duck Brand Printe...|          3|            0|          0|   N| 2014-09-21|
|   47596134|R103LVNJ2V5MOU|B002MJJ3EQ|      69208640|Delta T14485-SSH2...|        

In [ ]:
#Examine the dataframe schema to check the datatypes

cleaned_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#Updating the schema to have nullable = False

cleaned_df.schema['customer_id'].nullable=False
cleaned_df.schema['review_id'].nullable=False
cleaned_df.schema['product_id'].nullable=False

print(cleaned_df.schema['customer_id'])
print(cleaned_df.schema['review_id'])
print(cleaned_df.schema['product_id'])

StructField(customer_id,IntegerType,false)
StructField(review_id,StringType,false)
StructField(product_id,StringType,false)


### Create Dataframe for each of the Tables

In [ ]:
# Create review_id dataframe to match review_id_table table
review_id_df = cleaned_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100D3UT1KL2LP|    1297897|B002N1U1FI|     414800992| 2014-09-19|
|R100VJOWSRD7MW|   25517286|B000NCV2V6|     577225575| 2013-11-24|
|R103699E1EG79Y|   16192717|B002TSN4SQ|     786521714| 2015-06-01|
|R1038R168TQ0M2|    4583604|B007V4YAOK|     111253254| 2014-09-21|
|R103LVNJ2V5MOU|   47596134|B002MJJ3EQ|      69208640| 2013-12-16|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Create products dataframe to match products table
products_df = cleaned_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002N1U1FI|Wallies 12238 Cam...|
|B000NCV2V6|Heath Zenith BL-6...|
|B002TSN4SQ|Master Lock 1500I...|
|B007V4YAOK|Duck Brand Printe...|
|B002MJJ3EQ|Delta T14485-SSH2...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
print(f"Size of products before removing any duplicates: {products_df.count()}")

Size of products before removing any duplicates: 2634255


In [ ]:
# Dropping Duplicate values from the Dataframe
products_df = products_df.dropDuplicates()

# State the size of the DF after duplicates:
print(f"Size AFTER removing any duplicates: {products_df.count()}")

Size AFTER removing any duplicates: 348799


In [ ]:
#Create customers dataframe to match customers table and calculate the customer_count
customers_df = cleaned_df.groupby("customer_id").agg({"customer_id": "count"})

# Rename the column to match the table schema
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   22055470|            12|
|   53041874|             6|
|    3951073|             2|
|   20786504|             3|
|   47973619|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
# Create vine dataframe to match vine_table table
vine_df = cleaned_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100D3UT1KL2LP|          5|            0|          0|   N|
|R100VJOWSRD7MW|          4|            0|          0|   N|
|R103699E1EG79Y|          5|            0|          0|   N|
|R1038R168TQ0M2|          3|            0|          0|   N|
|R103LVNJ2V5MOU|          1|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



## Load

1.   Postgres setup
2.   Write Dataframes to RDS

### Postgres Setup

In [ ]:
# Configure settings for RDS - Please enter your endpoint and database information
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/<insert database-name>"
config = {"user":"<insert username>", 
          "password": "<insert password>", 
          "driver":"org.postgresql.Driver"}


### Write Dataframes to RDS

In [ ]:
# Write DataFrame to review_id_table table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)